In [10]:
import pandas as pd

In [11]:
data = pd.read_csv("Open_Ended_Cars.csv",header = 1)

In [12]:
len(data)

525

In [13]:
data["How old are you? "].median()

29.0

In [14]:
data["What was your household income in the last tax year, before tax was deducted? "].median()

6.0

In [15]:
def median_split(data, i):
    split = len(data)//2
    ordered = data.sort_values(by = i)
    return ordered.iloc[:split], ordered.iloc[split+1:]

In [16]:
lower_age, upper_age = median_split(data, "How old are you? ")
lower_income, upper_income = median_split(data, "What was your household income in the last tax year, before tax was deducted? ")

In [17]:
car_aliases = [['Jeep', 'Fiat', 'Chrysler'],
        ['Subaru', 'Bugeye', 'Scooby'],
        ['Dodge', 'Polara'],
#        ['GMC'],
        ['Tesla'],
#        ['Buick'],
        ['Toyota', 'Yota', 'Camry'],
        ['Honda', 'Accord'],
        ['Nissan', 'Infiniti'],
        ['Chevrolet', 'Chevy'],
        ['Hyundai', 'Tiburon', 'HYU', 'Kia'],
#        ['Ram'],
        ['Mazda', 'Matsuda'],
        ['Renault', 'Dacia'],
        ['Lamborghini', 'Lambo', 'Aventadora', 'Lamborgini'],
        ['Mercedes-Benz', 'Merc', 'Benz', 'Mercedes'],
        ['BMW', 'Beemer', 'Bimmer', 'Beamer'],
        ['Ford', 'Thunderbird', 'Mustang'],
        ['Porsche', 'Porche', 'Porce'],
        ['Audi', '4 Rings'],
        ['Volkswagen', 'VW', 'Volkswagon'],
        ['Ferrari', 'Prancing Horse', 'Scuderia'],
#        ['MG', 'M.G.', 'Morris Garages', 'M.G. Car Company'],
        ['Lexus', 'Lex', 'GX', 'ES 250'],
#        ['Infiniti'],
        ['Volvo']]

In [18]:
car_columns = ["Ford is similar to... ",
"Hyundai is similar to... ",
"BMW is similar to... ",
"Subaru is similar to... ",
"Jeep is similar to... ",
"Toyota is similar to... ",
"Chevrolet is similar to... ",
"Honda is similar to... ",
"audi",
"Mercedes-Benz is similar to... ",
"Tesla is similar to... ",
"Dodge is similar to... ",
"Lexus is similar to... ",
"Porsche is similar to... ",
"Volkswagen is similar to... ",
"Ferrari is similar to... ",
"Nissan is similar to... ",
"Mazda is similar to... ",
"Lamborghini is similar to... ",
"Renault is similar to... ",
"Volvo is similar to... "]

In [19]:
def column_count(data, aliases, prompt):
    alias_pattern = ""
    for i in aliases:
        alias_pattern += i
        alias_pattern += "|"
    alias_pattern = alias_pattern[:-1]   
    series = data[prompt].str.contains(alias_pattern)
    if True in series.unique():
        return series.value_counts()[True]
    return 0

In [20]:
def make_freq_table(data):
    counts = [[] for i in range(len(car_columns))]
    brands = []
    for aliases in car_aliases:
        brands.append(aliases[0])
        for i,prompt in zip(list(range(len(car_columns))), car_columns):
            count = column_count(data, aliases,prompt)
            counts[i].append(count)
    d = dict()
    d["brand"] = brands
    for i,j in zip(car_columns, counts):
        d[i] = j
    df = pd.DataFrame(data=d)
    return df

In [21]:
lower_age_counts = make_freq_table(lower_age)
upper_age_counts = make_freq_table(upper_age)
lower_income_counts = make_freq_table(lower_income)
upper_income_counts = make_freq_table(upper_income)

In [30]:
def most_similar(df):
    return df.set_index("brand").idxmax(axis =1)
most_similar(lower_income_counts)

brand
Jeep                     Dodge is similar to... 
Subaru                   Volvo is similar to... 
Dodge                     Jeep is similar to... 
Tesla                                       audi
Toyota                   Honda is similar to... 
Honda                   Toyota is similar to... 
Nissan                   Mazda is similar to... 
Chevrolet                 Ford is similar to... 
Hyundai                Hyundai is similar to... 
Mazda                    Honda is similar to... 
Renault                Renault is similar to... 
Lamborghini            Ferrari is similar to... 
Mercedes-Benz              BMW is similar to... 
BMW              Mercedes-Benz is similar to... 
Ford                 Chevrolet is similar to... 
Porsche            Lamborghini is similar to... 
Audi                       BMW is similar to... 
Volkswagen               Volvo is similar to... 
Ferrari            Lamborghini is similar to... 
Lexus            Mercedes-Benz is similar to... 
Volvo         

In [57]:
#brand# #low inc# #high# #same?#
def similarity_table(df_low, df_high):
    low = most_similar(df_low)
    high = most_similar(df_high)
    pd_dict = {"Low":low, "High":high}
    df =  pd.DataFrame(pd_dict)
    df["Low"] = df["Low"].str.split().str[0]
    df["High"] = df["High"].str.split().str[0]
    df["equal"] = df["Low"] == df["High"]
    return df

In [58]:
income_similarity = similarity_table(lower_income_counts, upper_income_counts)
age_similarity = similarity_table(lower_age_counts, upper_age_counts)

In [59]:
def diff_only(sim_table):
    df = sim_table[~sim_table["equal"]]
    return df.drop(columns=["equal"])

In [60]:
income_diff = diff_only(income_similarity)
age_diff = diff_only(age_similarity)

In [61]:
income_diff

,Low,High
brand,,
Jeep,Dodge,Subaru
Subaru,Volvo,Mazda
Dodge,Jeep,Ford
Tesla,audi,Tesla
Mazda,Honda,Subaru
Renault,Renault,Volkswagen
Porsche,Lamborghini,Ferrari
Lexus,Mercedes-Benz,audi


In [62]:
age_diff

,Low,High
brand,,
Jeep,Dodge,Subaru
Subaru,Mazda,Volkswagen
Dodge,Jeep,Ford
Tesla,Lexus,Tesla
Hyundai,Nissan,Hyundai
Mazda,Subaru,Nissan
Renault,Renault,Volkswagen
Porsche,Lamborghini,Ferrari
Lexus,audi,Mercedes-Benz
